In [526]:
# import string
import pyinflect
import spacy
import re
nlp = spacy.load('en_core_web_sm')


In [407]:
def find_subject_plural(token):
    subjs = [t for t in token.lefts if 'subj' in t.dep_]
    # nouns = [t for t in token.lefts if 'NOUN' in t.dep_]
    nouns = [t for t in token.sent[:token.i] if 'NOUN' in t.pos_]
    if len(subjs) > 0 :
        # print("Subject found")
        # print(subjs)
        morph = subjs[0].morph.to_dict()
        if 'Number' in morph.keys():
            return morph['Number']
        elif 'S' in subjs[0].tag_:
            return 'Plur'
        else:
            return 'Sing'
            
            
    elif len(nouns) > 0 :
        # print("Nouns found")
        # print(nouns)
        morph = nouns[0].morph.to_dict()
        if 'Number' in morph.keys():
            return morph['Number']
        elif 'S' in nouns[0].tag_:
            return 'Plur'
        else:
            return 'Sing'
    
    else:
        return 'Sing'

        
def sentence_past_tense(txt, nlp):
    doc = nlp(txt)
    out = list()
    words = []
    aux_count = 0
    aux_phrase = []

    
    for word in doc:
        words.append(word)

        tag = word.tag_
        pos = word.pos_
        dep = word.dep_

        
        if 'AUX' in pos and dep != 'ROOT':
            aux_count = aux_count +1
            aux_phrase.append(word.lemma_)
            
        else:  
            if dep == "ROOT" or 'cl' in dep:
                print("Root:"+ word.text)
                print(aux_phrase)
                if (('be' in aux_phrase) or ('am' in aux_phrase) ) and aux_count > 1:
                    
                    plural = find_subject_plural(word)
                    print(word.sent[word.i-2])
                    if 'RB' in word.sent[word.i-2].tag_:

                        print('ADVERBBBB')
                        if plural == 'Sing':
                            out.insert(word.i-1, 'was')
                        else:
                            out.insert(word.i-1, 'were')
                    else:
                        # print(plural)
                        print('not adverb')
                        if plural == 'Sing':
                            out.append('was')
                        else:
                            out.append('were')
                    
                    aux_count = 0
                    aux_phrase = []
                elif (('be' in aux_phrase) or ('am' in aux_phrase) ) and word.tag_ == 'VBN':
                    # print('huh')
                    plural = find_subject_plural(word)
                    # print(plural)
                    if plural == 'Sing':
                        out.append('was')
                    else:
                        out.append('were')
                    aux_count = 0
                    aux_phrase = []

                if tag in ['VBN', 'VBP', 'VBZ']:
                    plural = find_subject_plural(word)
                    print(word.text +' verb conjugation')
                    if plural == 'Sing':
                        new_word = word._.inflect('VBD', form_num = 0)
                    else:
                        new_word = word._.inflect('VBD', form_num = 1) 
                else:
                    new_word = None
                

                if new_word is not None:
                    out.append(new_word)
                else:
                    out.append(word.text)
                aux_count = 0
                aux_phrase = []
            
            else:
                out.append(word.text)
    
    # print(out)
    if len(out) > 0:
        out = " ".join(out)
            
    return out
    
  

def tense_correct_para(txt):
    nlp = spacy.load('en_core_web_sm')
    delimiters = r'[.:;\t]'
    chunks = re.split(delimiters, txt)
    for c in chunks:
        new_chunk = sentence_past_tense(c, nlp)
        print(c)
        if len(new_chunk) > 0:
            # print(new_chunk)
            txt = txt.replace(c, new_chunk)
    

    return txt

        
        

# will aux - drop
# will be aux - was/were
# will have - drop
# are going to - drop
# are - drop
        
# accounts can be updated - accounts were updated

    
    

In [389]:
txt = "•	Blinding of Study: The study will not be blinded"

In [390]:
txt2 = tense_correct_para(txt)

Root:•
[]
•
Root:Blinding
[]
Blinding of Study
Root:blinded
['will', 'be']
Nouns found
[study]
be
not adverb
Nouns found
[study]
blinded verb conjugation
 The study will not be blinded


In [391]:
test = nlp(txt)

In [392]:
[print(a) for a in test[-1].children if 'RB' in a.tag_]

not


[None]

In [393]:
test = nlp("For n=5 per group, brains will be dissected out, split into left and right hemispheres, placed in individual cryovials, and then frozen in isopentane pre-chilled on dry ice")

In [528]:
test = nlp('Here, we have a sentences')

In [537]:
all_words = [" "+token.text if token.pos_ != 'PUNCT' else token.text for token in test]

In [540]:
type(test[0])

spacy.tokens.token.Token

In [524]:
def find_subject_plural(token):
    # subjs = [t for t in token.ancestors if 'subj' in t.dep_]
    subjs = [t for t in token.lefts if 'subj' in t.dep_]
    # nouns = [t for t in token.lefts if 'NOUN' in t.dep_]
    nouns = [t for t in token.sent[:token.i] if 'NOUN' in t.pos_]
    if subjs:

        # print(subjs[0])
        morph = subjs[0].morph.to_dict()
        if 'Number' in morph.keys():
            return morph['Number']
        elif 'S' in subjs[0].tag_:
            return 'Plur'
        else:
            return 'Sing'
            
            
    elif nouns:
        # print(nouns[0])
        morph = nouns[-1].morph.to_dict()
        if 'Number' in morph.keys():
            return morph['Number']
        elif 'S' in nouns[-1].tag_:
            return 'Plur'
        else:
            return 'Sing'
    
    else:
        return 'Sing'

def past_tensify(txt, nlp):
    doc = nlp(txt)
    out = list()
    words = []
    aux_count = 0
    aux_phrase = []

    for word in doc:
        words.append(word)
        new_word = None
        tag = word.tag_
        pos = word.pos_
        dep = word.dep_

        if 'AUX' in pos and dep != 'ROOT' and out[-1].lower() != 'to':# and word.text != 'is':
            aux_count = aux_count +1
            aux_phrase.append(word.lemma_)

            # remove auxillary words, but keep track of recent removals. 
            # Do not remove auxillary words that are roots (i.e. be as the root verb)
            # Do not remove auxillary words following 'to'
            
        else:  
            if (dep == "ROOT") or ('cl' in dep) or ('conj' in dep) :

                if dep == 'ROOT':
                    print("ROOT: "+word.text)
                
                if 'cl' in dep:
                    print("cl: "+word.text)

                if 'conj' in dep:
                    print('conj: '+word.text)
                
                # determine if the subject is plural or singular
                plural = find_subject_plural(word)

                # pull out any adverbs
                adverbs = [a for a in word.children if 'RB' in a.tag_]

                # Identify 'be' in auxillary clause (will be, have been) to replace with was/were
                # or Identify 'be' in conjuction with past participle (are having, am walking) to replace with was/were
                if ((('be' in aux_phrase) or ('am' in aux_phrase) ) and aux_count > 1) or \
                    ((('be' in aux_phrase) or ('am' in aux_phrase) ) and word.tag_ in ['VBN', 'VBG']): 
                    print('AUX lemmas \t')
                    print(aux_phrase)

                    if word.tag_ != 'VBG':
                        if plural == 'Sing':
                            out.append('was')
                        else:
                            out.append('were')
                    else:
                        new_word = word._.inflect('VBD', form_num = 0)
                
                    aux_count = 0
                    aux_phrase = []

                if (tag in ['VB', 'VBP']) and ( word.text.lower() != 'see') and (out[-1].lower() != 'to'): # removed VBN and VBZ in wild test # 

                    if word.lemma_ == 'be':
                        print('be lemma')
                        if plural == 'Sing':
                            
                            new_word = 'was'
                        else:
                            
                            new_word = 'were'
                    else:      
                        if plural == 'Sing':
                            new_word = word._.inflect('VBD', form_num = 0)

                            print('Inflection: '+ new_word)
                        else:
                            new_word = word._.inflect('VBD', form_num = 1)
                            print('Inflection: '+ new_word)

                elif new_word is None:
                    new_word = word.text
                
                for adv in adverbs:
                    print("ADVERB: "+adv.text)
                    if adv.text in out and adv.i in range(word.i - (len(adverbs)+1), word.i + (len(adverbs)+1)):
                        
                        out.pop(out.index(adv.text))
                        out.append(adv.text)

                if new_word is not None:
                    out.append(new_word)
                else:
                    out.append(word.text)
                
                aux_count = 0
                aux_phrase = []
            
            else:
                out.append(word.text)


            

    # print(out)
    if len(out) > 0:
        out = " ".join(out)

    return out     
  

In [510]:
errors = [
    "The stimulus will automatically shut off when the animal withdraws its paw and the latency to the withdrawal will be recorded, with a cutoff of 20 s employed to prevent tissue damage. ",
    "Here we are providing a formal proposal for a study involving 3 treatment groups.",
    "A small pilot dose response study to harmaline will be conducted in CD-1 mice to determine the dose of harmaline to be used in the main study.",
    "The heat stimulus will be adjusted prior to the experiment so as to obtain approximately 10 second baseline responses in control rats. ",
    "Body weight will be monitored daily and clinical score will be measured daily beginning at day 7 after immunization and ending on day 30.",
    "Mice at this stage are given food on the cage floor, long sipper tubes, and daily subcutaneous saline injections to prevent death by dehydration.",
    "*Note: EAE is cyclical, so about 21 days after immunization, some animals may begin to return to normal “0”. They will then begin the next cycle."
    ]

In [514]:
errs = ["Once the rats remain still, a radiant heat source will be moved beneath the portion of the ipsilateral paw that is flush with the glass. ",
        "IM injection volume will be 20 µl (=1 µg) per injections site into each of the left and right caudal thigh muscle. ",
        "The imaging time will be 1 min post- D-Luciferin injection. "
        ]

In [525]:
past_tensify(errs[-1],nlp)

ROOT: be
be lemma


'The imaging time was 1 min post- D - Luciferin injection .'

In [516]:
test = nlp(errs[-1])

In [519]:
test[4].tag_

'VB'

In [450]:
test = nlp(chunks[1])

In [479]:
test[2].lemma_

'be'